This is necessary because I'm using old Docker image compiled for different version: https://blog.ml6.eu/serving-decision-forests-with-tensorflow-b447ea4fc81c 

The `!` sign before commands is for Colab to run the terminal command. Disregard that in normal environment.

In [ ]:
!pip uninstall tensorflow -y
!pip install tensorflow==2.5.1 tensorflow_decision_forests==0.1.8 pandas wurlitzer

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
import tensorflow_decision_forests as tfdf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd

print('TensorFlow version: {}'.format(tf.__version__))
# should be 2.5.1

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

DATASET_DIR = "/content/gdrive/MyDrive/DATA/dataset.csv"

DATASET = pd.read_csv(DATASET_DIR)
# Display the first 3 examples.
DATASET.head(3)


In [ ]:
# Split the dataset into a training and a testing dataset.
def split_dataset(dataset, test_ratio=0.20):
  """Splits a panda dataframe in two."""
  test_indices = np.random.rand(len(dataset)) < test_ratio
  return dataset[~test_indices], dataset[test_indices]

train_ds_pd, test_ds_pd = split_dataset(DATASET)
print("{} examples in training, {} examples for testing.".format(
    len(train_ds_pd), len(test_ds_pd)))


Convert pandas dataframe into tensorflow dataset

In [ ]:
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label="is_phishing")
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_ds_pd, label="is_phishing")


Train the model
Note:
No input features are specified. Therefore, all the columns will be used as input features except for the label. The feature used by the model are shown in the training logs and in the model.summary()

In [ ]:
# Specify the model.
model = tfdf.keras.RandomForestModel()

# Optionally, add evaluation metrics.
model.compile(
    metrics=["accuracy"])

# Train the model.
# "sys_pipes" is optional. It enables the display of the training logs. It doesn't work for some reason.
#with sys_pipes():
model.fit(x=train_ds)

  


In [ ]:
evaluation = model.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")


In [ ]:
#model.summary()
#model.make_inspector().variable_importances()
#model.make_inspector().evaluation()
model.make_inspector().features()



In [ ]:
tfdf.model_plotter.plot_model_in_colab(model, tree_idx=0, max_depth=3)


Default TF-serving produces error. Finally found why that's happening. The explanation:
>TensorFlow Serving is a serving system for TensorFlow models in production environments. The TF-Serving team publishes a pre-compiled release containing only Core TensorFlow ops.

>TensorFlow Decision Forests (TF-DF) models use custom Ops for inference. Therefore, they are not compatible with the pre-compiled releases of TF-Serving. The solution to serve TF-DF models with TF-Serving is to re-compile TF-Serving with TF-DF ops. This document explains how to do so.

https://github.com/tensorflow/decision-forests/blob/main/documentation/tensorflow_serving.md

In [ ]:
MODEL_DIR = "/content/gdrive/MyDrive/phishingModelAllUrlFeatures_v2"
# model_1.save("/content/gdrive/MyDrive/firstModelSave")
# Fetch the Keras session and save the model
# The signature definition is defined by the input and output tensors,
# and stored with the default serving key
#import tempfile

version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}
